In [0]:
spark.conf.set("fs.azure.account.key.datastorageaccountdummy.blob.core.windows.net", "s5C7IlqolmY+OrvxvLOrK1UL2ri0LyydTQqGaJQd+jjxfLANWkj153tsiG+cvRpxFakmTQ7NXFQM+AStsLYr+A==")

# dbutils.fs.unmount("/mnt/ingestion")

def is_mounted(mount_point):
    try:
        dbutils.fs.ls(mount_point)
        return True
    except Exception as e:
        if 'java.io.FileNotFoundException' in str(e):
            return False
        else:
            raise Exception(f"error: {str(e)}")

if not is_mounted("/mnt/ingestion"):
    dbutils.fs.mount(
        source = "wasbs://raw-data-ingestion@datastorageaccountdummy.blob.core.windows.net/",
        mount_point = "/mnt/ingestion",
        extra_configs = {"fs.azure.account.key.datastorageaccountdummy.blob.core.windows.net": "s5C7IlqolmY+OrvxvLOrK1UL2ri0LyydTQqGaJQd+jjxfLANWkj153tsiG+cvRpxFakmTQ7NXFQM+AStsLYr+A=="}
    )


In [0]:
def create_delta_table_if_not_exists(source_table_path,table,file_format,partition_columns):
    try:
        spark.sql("USE default") 
        df = spark.sql(f"""select * from {table}""")
        print("Delta table already exists.")
    except Exception as e:
        print("Delta table does not exist, creating it.")
        input_df = spark.read.format(file_format).load(source_table_path)
        if not partition_columns:
            input_df.write.format("delta").mode("overwrite").partitionBy(*partition_columns).saveAsTable(table)
        else:
            input_df.write.format("delta").mode("overwrite").saveAsTable(table)

source_table_path1 = "/mnt/ingestion/expedia/"
table1 = "expedia"
file_format1="avro"
partition_columns1=[]
source_table_path2="/mnt/ingestion/hotel-weather/"
table2="hotel_weather"
file_format2="parquet"
partition_columns2=["year","month","day"]
create_delta_table_if_not_exists(source_table_path1,table1,file_format1,partition_columns1)
create_delta_table_if_not_exists(source_table_path2,table2,file_format2,partition_columns2)


In [0]:
df_hotels=spark.sql("select * from expedia")
df_hotels.show()